# Table of Content

1. Import libraries and datasets
2. Grouping by Department id and Aggregating Data with mean of Order number
3. Creating customer groups (loyalty flag)
4. Analysis Customer Types
5. Creating spender groups
6. Creating frequent customer groups (frequency flag)
7. Exporting dataframe

# Import libraries and datasets

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os
import pickle

In [2]:
# Define path
path = r'/Users/frederikeschulz-mullensiefen/Desktop/Master Folder_Instacart/02_Data'

In [3]:
# Import orders_products_combined dataframe
df_ords_prods_merge = pd.read_pickle(os.path.join(path, 'Prepared Data', 'ords_prods_new_variables.pkl'))

In [4]:
# Creating a subset with 1000000 rows
df = df_ords_prods_merge[:1000000]

In [5]:
# Datacleaning: Replacing prices data of > 100 with nan 
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [6]:
# Datacleaning check 
df_ords_prods_merge['prices'].max()

25.0

# Grouping by Department id and Aggregating Data with mean of Order number

In [7]:
# Grouping subset by department_id and aggregating data with mean
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [8]:
# Grouping dataframe by department_id and aggregating data with mean
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


The results of the subset are different to the results of the whole dataframe. More specifically, in the subset, not all department_ids are included. Moreover, also the means differ for the department_ids. This is due to the fact that the subset does not include all observations: The first 1000000 rows do not include orders from all departments. And of course the mean changes as more rows (and data) are included in the whole dataframe.

# Creating customer groups (loyalty flag)

In [9]:
# Creating a column (max order) that groups the dataframe by the user_id and aggregates the data with the maximum order number
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/6r/_pj4fghj6jx_vml8b8yvngzh0000gn/T/ipykernel_16161/340936838.py:2: FutureWarning: The provided callable <function max at 0x10e485a80> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [10]:
# Displaying first 20 rows of dataframe
df_ords_prods_merge.head(20)

,order_id,user_id,order_number,orders_day_of_week,ordertime_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regular days,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Most orders,10
5,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regular days,Average orders,10
6,550135,1,7,1,9,20.0,196,1,1,Soda,77,7,9.0,Mid-range product,Busiest days,Most orders,10
7,3108588,1,8,1,14,14.0,196,2,1,Soda,77,7,9.0,Mid-range product,Busiest days,Most orders,10
8,2295261,1,9,1,16,0.0,196,4,1,Soda,77,7,9.0,Mid-range product,Busiest days,Most orders,10
9,2550362,1,10,4,8,30.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Average orders,10


In [11]:
# Creating conditions for loyal customers in a new loyalty_flag column based on max_order
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [12]:
# Creating conditions for regular customers in a new loyalty_flag column based on max_order
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [13]:
# Creating conditions for new customers in a new loyalty_flag column based on max_order
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [14]:
# Displaying first 20 rows of dataframe
df_ords_prods_merge.head(20)

,order_id,user_id,order_number,orders_day_of_week,ordertime_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Mid-range product,Regular days,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Average orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Most orders,10,New customer
5,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regular days,Average orders,10,New customer
6,550135,1,7,1,9,20.0,196,1,1,Soda,77,7,9.0,Mid-range product,Busiest days,Most orders,10,New customer
7,3108588,1,8,1,14,14.0,196,2,1,Soda,77,7,9.0,Mid-range product,Busiest days,Most orders,10,New customer
8,2295261,1,9,1,16,0.0,196,4,1,Soda,77,7,9.0,Mid-range product,Busiest days,Most orders,10,New customer
9,2550362,1,10,4,8,30.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days,Average orders,10,New customer


# Analysis Customer Types

In [15]:
# Grouping dataframe by customer types and aggregating data with mean, median, min, max 
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal customer    7.773575  1.0  25.0
New customer      7.801206  1.0  25.0
Regular customer  7.798262  1.0  25.0

New customers have the highest mean of 13.3 of product prices, suggesting that on average this customer group purchases more expensive products than the other two customer groups. Subsequently, regular customers have the second highest mean of 12.5 of product prices, followed by loyal customers, who have a mean of 10.4 of product prices. 
It is important to note, that this only gives an indication, as other statistical tests would be needed to see if there is a significant difference between these customer groups. It is also important to note, that this does not indicate which customer group is the most profitable one for Instacart.

# Creating spender groups 

In [16]:
# Creating a column (average price) that groups the dataframe by the user_id and aggregates the data with average price
df_ords_prods_merge['average_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

/var/folders/6r/_pj4fghj6jx_vml8b8yvngzh0000gn/T/ipykernel_16161/158565216.py:2: FutureWarning: The provided callable <function mean at 0x10e486480> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_ords_prods_merge['average_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [17]:
# Creating conditions for high spenders in a new spending_flag column based on avg price
df_ords_prods_merge.loc[df_ords_prods_merge['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [18]:
# Creating conditions for low spenders in a new spending_flag column based on avg price
df_ords_prods_merge.loc[df_ords_prods_merge['average_price'] < 10, 'spending_flag'] = 'Low spender'

In [19]:
# Checking frequency of spender groups
df_ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     32285131
High spender      119728
Name: count, dtype: int64

In [ ]:
freq_table = pd.crosstab(df_ords_prods_merge['region'], df_ordprodcust['spending_flag']) 

# Creating frequent customer groups (frequency flag)

In [29]:
# Creating a column (median_order_frequency) that groups the dataframe by the user_id and aggregates the data with median days_since_prior_order
df_ords_prods_merge['median_order_frequency'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

/var/folders/6r/_pj4fghj6jx_vml8b8yvngzh0000gn/T/ipykernel_17247/2666296881.py:2: FutureWarning: The provided callable <function median at 0x10a9a7ec0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_ords_prods_merge['median_order_frequency'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [30]:
# Creating conditions for frequent customers in a new order_frequency_flag column based on median days_since_prior_order
df_ords_prods_merge.loc[df_ords_prods_merge['median_order_frequency'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [31]:
# Creating conditions for regular customers in a new order_frequency_flag column based on median days_since_prior_order
df_ords_prods_merge.loc[(df_ords_prods_merge['median_order_frequency'] > 10) & (df_ords_prods_merge['median_order_frequency'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [32]:
# Creating conditions for non-frequent customers in a new order_frequency_flag column based on median days_since_prior_order
df_ords_prods_merge.loc[df_ords_prods_merge['median_order_frequency'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [33]:
# Checking frequency of frequency customer groups
df_ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
nan                             5
Name: count, dtype: int64

# Exporting dataframe

In [35]:
# Exporting dataframe
df_ords_prods_merge.to_pickle(os.path.join(path,'Prepared Data', 'ords_prods_flags.pkl'))